In [ ]:
# importing necessary libraries
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import os
import shutil

# Creating new folder to download weather data.....
if "Weather_Hourly_Data" not in os.listdir("."):
    os.makedirs("Weather_Hourly_Data")
else:
    shutil.rmtree(".\Weather_Hourly_Data")
    os.makedirs("Weather_Hourly_Data")

# Creating Date range for which weather data needs to be scrapped    
dates = pd.date_range(start="1-1-2018",end='30-4-2022',freq='D')
weather_hourly = pd.DataFrame()
dat = ""

for date in dates:
    try:
        # Converting date which is in Timestamp type to string type and extracting date from it
        dat = str(date).split()[0]    
        # URl from where hourly weather will be scrapped
        url = f"https://freemeteo.in/weather/washington/history/daily-history/?gid=4140963&station=19064&" \
                            f"date={dat}&language=english&country=us-united-states"

        page = requests.get(url)
        page_content = page.text

        soup = bs(page_content,"html.parser")

        # Extracting table from URl info using beautiful soup
        table = soup.findAll("div",{"class":"table hourly"})

        lis = table[0].text.strip().split("\n\n\n")

        # Organizing the scrapped data to create a dataframe from it
        for i in  range(1,len(lis)):
            lis[i] = lis[i].replace("\n","|")
            lis[i] = lis[i].replace("|| ","|")
            lis[i] = lis[i].lstrip("||")
            lis[i] = lis[i].split("|")

        # Creating dataframe for that date from above data
        dt = pd.DataFrame(lis[1:])
        # Adding Date column to the datafrme
        dt["date"] = dat

        print(f"Weather data collected for date : {dat} successfully")
        # Combining the daily data into single dataframe according to each year
        weather_hourly = pd.concat([weather_hourly,dt],axis=0)

        # Checking if its the last day of the year...if yes, then, convert merged data of each year into .csv file 
        if re.search("[\d]{4}-12-31",dat)==True or dat == "2022-04-30":
            columns = lis[0].split("\n")
            columns.remove("Icon")       # Removing unwanted column
            weather_hourly.columns = [*columns,"date"]   # Renaming the columns of Dataframe

            weather_hourly=weather_hourly.loc[:,["date", *columns]]    # Changing position of columns for better understanding

            # Converting Dataframe into .CSV file
            weather_hourly.to_csv(f".\Weather_Hourly_Data/{dat[0:4]}-weather-data.csv",index=False)
            print(f"Weather Data .csv file created for year : {dat[0:4]}")
            weather_hourly = pd.DataFrame() 

    except Exception as e:
        print("error at :", dat)
        print(e)